In [228]:
import requests
import numpy as np
import pandas as pd
import json
from IPython.display import display
import re
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from lib.twitter_keys import keys

In [229]:
sql_select = '''
select * from tweets_demo where lang = 'en'
'''


In [230]:
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='54.69.228.16'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
#cur.execute(sql_create)
#cur.execute(sql_drop)
#cur.execute(sql_insert)
#conn.commit()
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)

In [231]:
len(df)

65363

In [215]:
list(df.columns.values)

['business',
 'cleaned_tweet',
 'climate',
 'country',
 'created_at',
 'entities',
 'geo',
 'geo_enabled',
 'govt',
 'hashtags',
 'id',
 'immigrant',
 'lang',
 'latitude',
 'location',
 'longitude',
 'movies',
 'music',
 'place_type',
 'regin',
 'retweet_count',
 'retweeted',
 'screen_name',
 'sports',
 'tech',
 'time_zone',
 'trump',
 'tv',
 'tweet_content',
 'usr',
 'vetcor']

In [15]:
import pickle
!pip install redis
import redis
redis_ip = '52.37.247.211'
r = redis.StrictRedis(redis_ip)
#r.flushall()

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
def c(text):
    text = re.sub("'","''", text)
    text = re.sub("{","\{",text)
    text = re.sub("}","\}",text)
    return text

### Nearest Neighbors to Categorize Tweets

In [33]:
from spacy.en import English
nlp = English()

In [34]:
content_vecs = np.array([nlp(i).vector for i in df['tweet_content']])
content_vecs.shape

(65363, 300)

In [252]:
type(content_vecs[0])

numpy.ndarray

In [173]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=6500)
nn.fit(content_vecs)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=6500, p=2, radius=1.0)

In [174]:
def most_similar(search):
    distance, indices = nn.kneighbors(nlp(search).vector.reshape(1,-1))
    
    return df.ix[indices[0]][['location','created_at','tweet_content','trump']]

In [175]:
cat_sport = most_similar('sports')
cat_sport

,location,created_at,tweet_content,trump
20706,"south gate, ca",Thu Jun 01 08:12:16 +0000 2017,radio soccer radio fucken dirtbags soccer only,False
36600,"garden grove, ca",Thu Jun 01 18:38:25 +0000 2017,basketball is like poetry,False
48181,"montebello, ca",Thu Jun 01 21:03:53 +0000 2017,ughhh i need a sports bra,False
6164,"california, usa",Thu Jun 01 02:31:22 +0000 2017,@lilcookimonster never make excuses for sports,False
38492,"lynwood, ca",Thu Jun 01 19:02:54 +0000 2017,football season wya ?,False
38456,"los angeles, ca",Thu Jun 01 19:02:30 +0000 2017,@bleacherreport is the king of sports media 😂😂...,False
33827,"los angeles, ca",Thu Jun 01 17:34:34 +0000 2017,arena football league partners with twitter on...,False
38090,"west hollywood, ca",Thu Jun 01 18:57:35 +0000 2017,@kristineleahy is becoming my favorite reporte...,False
7154,"los angeles, ca",Thu Jun 01 02:51:24 +0000 2017,we at my bro's baseball game &amp; we talking ...,False
6587,"los angeles, ca",Thu Jun 01 02:38:48 +0000 2017,the penguins are the worst franchise in the hi...,False


In [176]:
cat_music = most_similar('music')
cat_music

,location,created_at,tweet_content,trump
5400,santa monica pier,Thu Jun 01 02:15:41 +0000 2017,listening to this guy and his jazz music,False
25,"los angeles, ca",Wed May 31 23:54:52 +0000 2017,christmas music all day,False
48756,"compton, ca",Thu Jun 01 21:11:44 +0000 2017,i hate country music,False
43253,"simi valley, ca",Thu Jun 01 20:01:39 +0000 2017,playing music is so fun,False
986,"los angeles, ca",Thu Jun 01 00:13:50 +0000 2017,we only listen to dance music murder rap or t...,False
5450,"california, usa",Thu Jun 01 02:16:52 +0000 2017,i need more music,False
45795,"los angeles, ca",Thu Jun 01 20:33:32 +0000 2017,4 the best @all4themusiccom in social media f...,False
18750,"california, usa",Thu Jun 01 06:59:03 +0000 2017,downloaded so much music rn🤘🏼,False
31013,"los angeles, ca",Thu Jun 01 16:54:12 +0000 2017,music was missing ☝️,False
47054,"chino, ca",Thu Jun 01 20:49:54 +0000 2017,bbq music. https://t.co/ldv6ptl2dd,False


In [177]:
cat_movies = most_similar('movies')
cat_movies

,location,created_at,tweet_content,trump
53714,"los angeles, ca",Thu Jun 01 22:21:00 +0000 2017,a movie https://t.co/jmaled1dgf,False
21947,"south gate, ca",Thu Jun 01 09:51:58 +0000 2017,xzibit lil black baseball comedy movie,False
56511,"los angeles, ca",Thu Jun 01 22:59:28 +0000 2017,love this movie https://t.co/xnfsq5gnsy,False
36658,"lakewood, ca",Thu Jun 01 18:39:19 +0000 2017,movie ruined now https://t.co/1odoyvipvr,False
22474,"california, usa",Thu Jun 01 10:54:11 +0000 2017,the lego batman movie https://t.co/pxmvbdvgmw,False
18949,"california, usa",Thu Jun 01 07:04:12 +0000 2017,the lego batman movie https://t.co/9vezx89t3u,False
17712,"california, usa",Thu Jun 01 06:31:04 +0000 2017,the lego batman movie https://t.co/hcrsitxe2z,False
1486,"california, usa",Thu Jun 01 00:24:44 +0000 2017,in need of some good tv shows/movies on netflix,False
8368,"los angeles, ca",Thu Jun 01 03:16:05 +0000 2017,i just wanna smoke and watch movies,False
55503,"california, usa",Thu Jun 01 22:45:27 +0000 2017,@onlyfilmmedia best arnold movie imo,False


In [178]:
cat_travel = most_similar('travel')
cat_travel

,location,created_at,tweet_content,trump
30216,"california, usa",Thu Jun 01 16:42:47 +0000 2017,travel and home https://t.co/cda1ffybar,False
62142,"california, usa",Fri Jun 02 00:16:05 +0000 2017,sleeping is like the cheapest form of time travel,False
40722,"fountain valley, ca",Thu Jun 01 19:31:35 +0000 2017,ready to travel with my boo😘 @kelly_gauss,False
57781,"california, usa",Thu Jun 01 23:17:09 +0000 2017,let's get tattoos and travel the world,False
33099,"los angeles, ca",Thu Jun 01 17:24:34 +0000 2017,i started a new ig account dedicated to travel...,False
29216,"chino hills, ca",Thu Jun 01 16:28:00 +0000 2017,i want to be an intern for a travel group in e...,False
37025,"cypress, ca",Thu Jun 01 18:44:13 +0000 2017,@victoriajuless turn around trip,False
52480,"los angeles, ca",Thu Jun 01 22:03:11 +0000 2017,i just want to travel and eat filtered culture...,False
36445,"california, usa",Thu Jun 01 18:36:33 +0000 2017,@latimes free bus tickets from the east,False
60055,"huntington beach, ca",Thu Jun 01 23:49:11 +0000 2017,what are good places for weekend trips,False


In [179]:
cat_TV = most_similar('TV')
cat_TV 

,location,created_at,tweet_content,trump
42932,"la habra, ca",Thu Jun 01 19:57:56 +0000 2017,my new tv sick af,False
1486,"california, usa",Thu Jun 01 00:24:44 +0000 2017,in need of some good tv shows/movies on netflix,False
41110,"anaheim, ca",Thu Jun 01 19:36:11 +0000 2017,i'm getting tv package https://t.co/hg5uob3s07,False
15165,"los angeles, ca",Thu Jun 01 05:35:05 +0000 2017,i'm officially into reality tv shows!,False
60095,"los angeles, ca",Thu Jun 01 23:49:49 +0000 2017,get sage steele off my tv,False
15128,"los angeles, ca",Thu Jun 01 05:34:21 +0000 2017,@syxxwilliams agreed i'm going live on the tv ...,False
21188,"los angeles, ca",Thu Jun 01 08:41:56 +0000 2017,sometimes i watch tv with my eyes closed.,False
21191,"los angeles, ca",Thu Jun 01 08:42:18 +0000 2017,sometimes i watch tv with my eyes closed.,False
56397,"los angeles, ca",Thu Jun 01 22:57:57 +0000 2017,gonna stay home and watch netflix tonight 💘,False
21172,"south gate, ca",Thu Jun 01 08:41:14 +0000 2017,anyways uiehl radio radio radios,False


In [180]:
cat_tech = most_similar('technology')
cat_tech

,location,created_at,tweet_content,trump
31431,"california, usa",Thu Jun 01 17:00:40 +0000 2017,business of technology final today,False
43071,"los angeles, ca",Thu Jun 01 19:59:34 +0000 2017,to understand about the telecommunications ind...,False
45746,"la crescenta-montrose, ca",Thu Jun 01 20:32:55 +0000 2017,"thanks to advance technology, ha is now improv...",False
12759,"commerce, ca",Thu Jun 01 04:48:18 +0000 2017,gotta use all your most advanced techniques. h...,False
31737,"irvine, ca",Thu Jun 01 17:05:20 +0000 2017,high stakes research: testing your #mobile soc...,False
22756,"los angeles, ca",Thu Jun 01 11:37:41 +0000 2017,coventry university launches national transpor...,False
41590,"los angeles, ca",Thu Jun 01 19:42:03 +0000 2017,the technology of the alphabet corporation can...,False
29373,"los angeles, ca",Thu Jun 01 16:30:12 +0000 2017,"""choice systems privilege parents with resourc...",False
45362,"los angeles, ca",Thu Jun 01 20:27:46 +0000 2017,coal is only an appropriate energy solution fo...,False
23879,"california, usa",Thu Jun 01 13:32:57 +0000 2017,residents-take advantage of this amazing techn...,False


In [181]:
cat_health = most_similar('health')
cat_health 

,location,created_at,tweet_content,trump
27349,"beverly hills, ca",Thu Jun 01 15:39:01 +0000 2017,it’s time to recognize mental health as essent...,False
65074,"santa monica, ca",Thu Jun 01 22:45:39 +0000 2017,the effects of climate change to human health....,False
58192,"los angeles, ca",Thu Jun 01 23:22:31 +0000 2017,working in mental health has humbled me more t...,False
54151,"los angeles, ca",Thu Jun 01 22:27:26 +0000 2017,time to eat healthy and remember that food is ...,False
14773,"los angeles, ca",Thu Jun 01 05:27:20 +0000 2017,nuhw and kaiser permanente: collaborating to i...,False
3552,"los angeles, ca",Thu Jun 01 01:36:00 +0000 2017,@altamonttt @mitchellreports tried to provide ...,False
35624,"los angeles, ca",Thu Jun 01 18:25:55 +0000 2017,from health care to climate change.... there's...,False
48275,"los angeles, ca",Thu Jun 01 21:05:02 +0000 2017,@gtorres25 @attilioarmeni heaven forbid we do ...,False
22810,"los angeles, ca",Thu Jun 01 11:43:56 +0000 2017,stay consistent in your training and in your n...,False
37536,"beverly hills, ca",Thu Jun 01 18:50:45 +0000 2017,"dialysis providers, akf program would benefit ...",False


In [182]:
cat_food = most_similar('food')
cat_food

,location,created_at,tweet_content,trump
38837,churrasco (brazilian style bbc),Thu Jun 01 19:07:03 +0000 2017,brazilian food for lunch 👌,False
45303,"west hollywood, ca",Thu Jun 01 20:27:08 +0000 2017,just eat some food. https://t.co/2rpgjeex6t,False
42713,"anaheim, ca",Thu Jun 01 19:57:50 +0000 2017,chinese food may be too good,False
3103,"los angeles, ca",Thu Jun 01 01:26:32 +0000 2017,or damn just eat some food https://t.co/pqrhrd...,False
18120,"alhambra, ca",Thu Jun 01 06:42:31 +0000 2017,need some fucking food,False
26495,"los angeles, ca",Thu Jun 01 15:14:15 +0000 2017,@angelanissel food for less?,False
30880,"whittier, ca",Thu Jun 01 16:52:26 +0000 2017,ya'll making food to eat it? https://t.co/mgun...,False
16323,"los angeles, ca",Thu Jun 01 05:59:24 +0000 2017,okay no more junk food for me🙅🏽‍♂️,False
12377,"los angeles, ca",Thu Jun 01 04:40:48 +0000 2017,@heosekki i only eat meat,False
18726,"los angeles, ca",Thu Jun 01 06:58:20 +0000 2017,do receptions include food?,False


In [183]:
cat_business = most_similar('busniess')
cat_business

,location,created_at,tweet_content,trump
50662,"manhattan beach, ca",Thu Jun 01 21:38:43 +0000 2017,preach https://t.co/ovzqg396kg,False
44240,"los angeles, ca",Thu Jun 01 20:13:33 +0000 2017,@baylorthegreat @jusblaze_513 @brotherslehman ...,False
58068,"compton, ca",Thu Jun 01 23:21:02 +0000 2017,extraaaaa pineapple pleease https://t.co/bsm4q...,False
43357,"santa ana, ca",Thu Jun 01 20:02:43 +0000 2017,@dane_guy @adultbrazil @biencalmex @erotictemp...,False
44599,"pomona, ca",Thu Jun 01 20:18:05 +0000 2017,@kaptainkamm retweet,False
5767,"los angeles, ca",Thu Jun 01 02:23:23 +0000 2017,@kxxxyxng_ showny,False
19145,"los angeles, ca",Thu Jun 01 07:09:25 +0000 2017,june,False
5064,"los angeles, ca",Thu Jun 01 02:08:14 +0000 2017,@mmafa @cernovich @muhammadali @repmullin @jon...,False
49542,"walnut, ca",Thu Jun 01 21:22:27 +0000 2017,@brunomars chunkyyyy,False
28893,"los angeles, ca",Thu Jun 01 16:20:03 +0000 2017,if👏🏽this👏🏽ain't👏🏽me👏🏽at👏🏽work👏🏽 https://t.co/h...,False


In [184]:
cat_gov = most_similar('government')
cat_gov 

,location,created_at,tweet_content,trump
56395,"california, usa",Thu Jun 01 22:57:56 +0000 2017,provincial government electrical monopoly bull...,False
21971,"south gate, ca",Thu Jun 01 09:54:29 +0000 2017,hmmh or a related social government movie the ...,False
44583,"california, usa",Thu Jun 01 20:17:54 +0000 2017,@elonmusk you make ur money on slave workers i...,False
34081,"los angeles, ca",Thu Jun 01 17:37:38 +0000 2017,why should the average american abide by the l...,False
64744,"california, usa",Thu Jun 01 23:41:36 +0000 2017,@billnye trump's announcement will trigger the...,True
64352,"los angeles, ca",Thu Jun 01 21:17:57 +0000 2017,president trump is a champion for hard-working...,True
6996,"los angeles, ca",Thu Jun 01 02:48:15 +0000 2017,treason is crime under federal and some state ...,False
6678,"east los angeles, ca",Thu Jun 01 02:40:32 +0000 2017,area president still trying to convince americ...,False
13491,"california, usa",Thu Jun 01 05:02:28 +0000 2017,@ancorch employers are already breaking the la...,False
65333,"los angeles, ca",Thu Jun 01 23:04:30 +0000 2017,city and state officials commit to tackling th...,True


In [185]:
cat_law = most_similar('law')
cat_law

,location,created_at,tweet_content,trump
34081,"los angeles, ca",Thu Jun 01 17:37:38 +0000 2017,why should the average american abide by the l...,False
6996,"los angeles, ca",Thu Jun 01 02:48:15 +0000 2017,treason is crime under federal and some state ...,False
30188,"diamond bar, ca",Thu Jun 01 16:42:23 +0000 2017,geez @joerogan research the current law suit a...,False
53077,"south whittier, ca",Thu Jun 01 22:11:53 +0000 2017,law of attraction 🤘🏾 https://t.co/8hfhjnvusb,False
28101,"california, usa",Thu Jun 01 15:58:58 +0000 2017,what should local law enforcement agencies do ...,False
13491,"california, usa",Thu Jun 01 05:02:28 +0000 2017,@ancorch employers are already breaking the la...,False
7416,"california, usa",Thu Jun 01 02:56:41 +0000 2017,old there any widow law rooms,False
44053,"venice, los angeles",Thu Jun 01 20:11:20 +0000 2017,there is no tax bill https://t.co/tsse9cj6dv,False
34560,"california, usa",Thu Jun 01 17:44:35 +0000 2017,residence certainly elsewhere something she pr...,False
23031,"los angeles, ca",Thu Jun 01 12:16:45 +0000 2017,the ada has been law for 27 years. it is no se...,False


In [186]:
cat_education= most_similar('education')
cat_education

,location,created_at,tweet_content,trump
47176,"anaheim, ca",Thu Jun 01 20:51:13 +0000 2017,attending national labor employment associatio...,False
23221,"california, usa",Thu Jun 01 12:39:38 +0000 2017,survey finds regrets among most former college...,False
61988,"los angeles, ca",Fri Jun 02 00:14:11 +0000 2017,apparently his grad school grammar/spelling ed...,False
44587,"california, usa",Thu Jun 01 20:17:57 +0000 2017,being an ag science student during an animal e...,False
4365,"los angeles, ca",Thu Jun 01 01:54:00 +0000 2017,science &amp; american leadership and particip...,False
21153,"california, usa",Thu Jun 01 08:40:25 +0000 2017,@fasabutt @morphegrl @yhcniv @dshook7 zoos sti...,False
38730,"los angeles, ca",Thu Jun 01 19:05:38 +0000 2017,high school besties✨ https://t.co/4wrakh2n04,False
38064,"long beach, ca",Thu Jun 01 18:57:10 +0000 2017,my baby seniors graduate high school today. 😭😭😭,False
29813,"los angeles, ca",Thu Jun 01 16:36:41 +0000 2017,@peddoc63 @haislopvictor they also want to con...,False
42120,"anaheim, ca",Thu Jun 01 19:48:45 +0000 2017,@glittergeekali then give these people free ed...,False


In [187]:
cat_entertainment = most_similar('entertainment')
cat_entertainment

,location,created_at,tweet_content,trump
45795,"los angeles, ca",Thu Jun 01 20:33:32 +0000 2017,4 the best @all4themusiccom in social media f...,False
38456,"los angeles, ca",Thu Jun 01 19:02:30 +0000 2017,@bleacherreport is the king of sports media 😂😂...,False
35320,"los angeles, ca",Thu Jun 01 18:21:33 +0000 2017,we've got your comprehensive guide to the best...,False
56339,"los angeles, ca",Thu Jun 01 22:57:04 +0000 2017,shopping. restaurants. entertainment (@ westwo...,False
17326,"los angeles, ca",Thu Jun 01 06:21:04 +0000 2017,creative marketing/production at its finest 👌🏼...,False
57199,delta sky club terminal 3,Thu Jun 01 23:09:54 +0000 2017,today's entertainment comes by way of watching...,False
1486,"california, usa",Thu Jun 01 00:24:44 +0000 2017,in need of some good tv shows/movies on netflix,False
39957,"los angeles, ca",Thu Jun 01 19:21:36 +0000 2017,we in here. get your tickets for my big celebr...,False
37505,"burbank, ca",Thu Jun 01 18:50:18 +0000 2017,@gorburger is live at @clusterfest tomorrow @c...,False
4618,mayan theater,Thu Jun 01 01:58:53 +0000 2017,"music therapy night , at the mayan theater to ...",False


In [188]:
cat_fashion = most_similar('fashion')
cat_fashion

,location,created_at,tweet_content,trump
4884,"los angeles, ca",Thu Jun 01 02:04:40 +0000 2017,@asapferg fashion zaddy,False
33033,"beverly hills, ca",Thu Jun 01 17:23:37 +0000 2017,#juneisfinnabe a fashion revamp f,False
7626,"los angeles, ca",Thu Jun 01 03:00:44 +0000 2017,this is what i call fashion https://t.co/oqvkc...,False
3787,"los angeles, ca",Thu Jun 01 01:41:30 +0000 2017,enjoying designer jewelry thanks to @joinswitc...,False
37228,"los angeles, ca",Thu Jun 01 18:46:51 +0000 2017,trend setter,False
8772,"los angeles, ca",Thu Jun 01 03:25:24 +0000 2017,there is a hollywood fashion icon who has aban...,False
3709,"los angeles, ca",Thu Jun 01 01:39:52 +0000 2017,so in fashion it's okay for men to rock purses...,False
21538,"california, usa",Thu Jun 01 09:10:32 +0000 2017,my makeup inspiration 🙌🏼🤤💄 https://t.co/yawseo...,False
44442,"california, usa",Thu Jun 01 20:16:00 +0000 2017,"10. fashion wise, i tend to gravitate towards ...",False
27337,"los angeles, ca",Thu Jun 01 15:38:45 +0000 2017,"in the may issue, the fashion designer juan.ca...",False


In [189]:
cat_shopping = most_similar('shopping')
cat_shopping

,location,created_at,tweet_content,trump
17709,"california, usa",Thu Jun 01 06:31:02 +0000 2017,shopping? https://t.co/k3cwp5ooub,False
56868,"garden grove, ca",Thu Jun 01 23:04:58 +0000 2017,let's go romper shopping,False
34949,"los angeles, ca",Thu Jun 01 17:49:41 +0000 2017,we gon shopping at tiffany,False
44157,"los angeles, ca",Thu Jun 01 20:12:39 +0000 2017,let's go shopping?,False
53297,"los angeles, ca",Thu Jun 01 22:15:05 +0000 2017,i wanna go shopping,False
1449,"los angeles, ca",Thu Jun 01 00:24:01 +0000 2017,online shopping is annoying cus i want everything,False
30077,discovery cube orange county,Thu Jun 01 16:40:48 +0000 2017,grocery shopping. @oakridgeschool #fieldtrip ...,False
1536,"east los angeles, ca",Thu Jun 01 00:25:48 +0000 2017,i just want to go shopping,False
47192,"alhambra, ca",Thu Jun 01 20:51:24 +0000 2017,dedicating my weekend to shopping for cancun.,False
35186,"los angeles, ca",Thu Jun 01 18:19:48 +0000 2017,love breakfast shopping while everyone else is...,False


In [190]:
cat_song = most_similar('song')
cat_song

,location,created_at,tweet_content,trump
47642,"los angeles, ca",Thu Jun 01 20:57:01 +0000 2017,song*,False
50043,"long beach, ca",Thu Jun 01 21:30:16 +0000 2017,highway song,False
24492,"los angeles, ca",Thu Jun 01 14:05:19 +0000 2017,this your song,False
13509,"california, usa",Thu Jun 01 05:02:45 +0000 2017,i love old songs,False
16363,"los angeles, ca",Thu Jun 01 06:00:13 +0000 2017,i love this song. https://t.co/cdmkcupz2x,False
50114,"monterey park, ca",Thu Jun 01 21:31:19 +0000 2017,this song is beautiful https://t.co/wtuo4ptayv,False
8502,"los angeles, ca",Thu Jun 01 03:19:30 +0000 2017,my theme song everyday i swear lyrics &amp; sa...,False
62435,"los angeles, ca",Fri Jun 02 00:20:31 +0000 2017,heard hyeoyeon's song omg............. girl?,False
48756,"compton, ca",Thu Jun 01 21:11:44 +0000 2017,i hate country music,False
49985,"los angeles, ca",Thu Jun 01 21:29:34 +0000 2017,it was jus a song,False


In [204]:
#nothing related come up# 
cat_trump = most_similar('trump')
cat_trump

,location,created_at,tweet_content,trump
50662,"manhattan beach, ca",Thu Jun 01 21:38:43 +0000 2017,preach https://t.co/ovzqg396kg,False
44240,"los angeles, ca",Thu Jun 01 20:13:33 +0000 2017,@baylorthegreat @jusblaze_513 @brotherslehman ...,False
58068,"compton, ca",Thu Jun 01 23:21:02 +0000 2017,extraaaaa pineapple pleease https://t.co/bsm4q...,False
43357,"santa ana, ca",Thu Jun 01 20:02:43 +0000 2017,@dane_guy @adultbrazil @biencalmex @erotictemp...,False
44599,"pomona, ca",Thu Jun 01 20:18:05 +0000 2017,@kaptainkamm retweet,False
5767,"los angeles, ca",Thu Jun 01 02:23:23 +0000 2017,@kxxxyxng_ showny,False
19145,"los angeles, ca",Thu Jun 01 07:09:25 +0000 2017,june,False
5064,"los angeles, ca",Thu Jun 01 02:08:14 +0000 2017,@mmafa @cernovich @muhammadali @repmullin @jon...,False
49542,"walnut, ca",Thu Jun 01 21:22:27 +0000 2017,@brunomars chunkyyyy,False
28893,"los angeles, ca",Thu Jun 01 16:20:03 +0000 2017,if👏🏽this👏🏽ain't👏🏽me👏🏽at👏🏽work👏🏽 https://t.co/h...,False


In [192]:
sum(cat_gov.index.isin(cat_law.index) == True)/6500

0.73569230769230765

In [193]:
sum(cat_music.index.isin(cat_movies.index) == True)/6500

0.41092307692307695

In [194]:
sum(cat_music.index.isin(cat_song.index) == True)/6500

0.43523076923076925

In [195]:
sum(cat_entertainment.index.isin(cat_song.index) == True)/6500

0.075846153846153841

In [196]:
sum(cat_TV.index.isin(cat_movies.index) == True)/6500

0.41030769230769232

In [197]:
sum(cat_gov.index.isin(cat_business.index) == True)/6500

0.0069230769230769233

In [156]:
cat_music.index.isin(cat_entertainment.index) == True

array([ True,  True, False, ...,  True, False, False], dtype=bool)

In [198]:
sum(cat_music.index.isin(cat_entertainment.index) == True)/6500

0.32553846153846155

In [160]:
sum(cat_movies.index.isin(cat_entertainment.index) == True)/5000

0.24160000000000001

In [199]:
sum(cat_fashion.index.isin(cat_entertainment.index) == True)/6500

0.3943076923076923

In [200]:
sum(cat_fashion.index.isin(cat_shopping.index) == True)/6500

0.31846153846153846

In [201]:
sum(cat_business.index.isin(cat_shopping.index) == True)/6500

0.0081538461538461539

In [202]:
sum(cat_business.index.isin(cat_education.index) == True)/6500

0.016769230769230769

In [203]:
sum(cat_education.index.isin(cat_gov.index) == True)/6500

0.54923076923076919

In [205]:
sum(cat_trump.index.isin(cat_entertainment.index) == True)/6500

0.044461538461538462

In [206]:
sum(cat_trump.index.isin(cat_gov.index) == True)/6500

0.0069230769230769233

In [208]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import redis
redis_ip = '34.210.97.79'
content_vecs = pickle.loads(r.get('nlp_content_vecs'))
cat_vecs = pickle.loads(r.get('nlp_cat_vecs'))
def Similarity_score(text):
    wiki_cat_df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_cat_collection.find({})))
    #cat_vecs = np.array([nlp(i).vector for i in wiki_cat_df['text']])
    r = redis.StrictRedis(redis_ip)
    cat_vecs = pickle.loads(r.get('nlp_cat_vecs'))
    similarity_score={}
    for j in range(len(cat_vecs)):
        similarity = cosine_similarity(cat_vecs[j].reshape(1,-1),nlp(text).vector.reshape(1,-1))
        similarity_score[(wiki_cat_df['category'][j])] = round(similarity[0][0],3)
        cs_s_df = pd.DataFrame.from_dict(similarity_score,orient='index')
        cs_s_df.columns = ['score']
        cs_s_df = cs_s_df.sort_values('score',ascending=False)
    return cs_s_df.head(3)

In [210]:
#for j in range(len(content_vecs)):
for j in range(20):
    similarity = cosine_similarity(content_vecs[j].reshape(1,-1),nlp('music').vector.reshape(1,-1))
    print(df['tweet_content'].iloc[j], round(similarity[0][0],3))

i'm such a weak ass pu$$y bitch 0.396
remember to get up and move today! move free live free! 0.488
taking a quick break from preparing charts to sing one of my favorite marvin sapp songs and… https://t.co/b6ebreumfg 0.594
don't let #hardball fool you. william f buckley jr was a racist chauvinist asshole. he was outspoken but rude and narrow-minded. 0.404
i wonder what his reaction to chivas winning 😂 https://t.co/whlvguum0a 0.447
glad today was wednesday already (@ oaktree capital management in los angeles, ca) https://t.co/blbz5b0eg8 0.394
join the kohl's corporation team! see our latest #job opening here: https://t.co/q1yzmo32am #retail #manteca, ca #hiring 0.341
the #energy encompassed within hip- hop is beyond the #audible #spectrum.  https://t.co/oohkj6t9cl 0.411
@collinsdw lol. sorry. you being from texas and all... i just assumed the worst. ;-) 0.438
la zoo today 🙈🐨🐭🦇🐼🦊 . . . . #beverlyhills90210 #losangeles #lifestyle #lazoo #picoftheday… https://t.co/d7kfwdd8ai 0.246
i was su

In [216]:
def c(text):
    text = re.sub("'","''", text)
    text = re.sub("{","\{",text)
    text = re.sub("}","\}",text)
    return text

In [250]:
from tqdm import tqdm
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='54.69.228.16'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)

for i in tqdm(range(len(df))):
    tweet = c(df['tweet_content'].iloc[i])
    similarity = cosine_similarity(content_vecs[i].reshape(1,-1),nlp('business').vector.reshape(1,-1))
    cur = conn.cursor()
    sql_update = '''
    update tweets_demo
    set business = '{}'
    where tweet_content = '{}';
    commit;
    '''.format(round(similarity[0][0],2),tweet)
    cur.execute(sql_update)
    conn.commit() 
    print(round(similarity[0][0],2), df['tweet_content'].iloc[i])
conn.close  


  0%|          | 10/65363 [00:00<23:36, 46.13it/s]

0.34 glad today was wednesday already (@ oaktree capital management in los angeles, ca) https://t.co/blbz5b0eg8
0.49 join the kohl's corporation team! see our latest #job opening here: https://t.co/q1yzmo32am #retail #manteca, ca #hiring
0.44 the #energy encompassed within hip- hop is beyond the #audible #spectrum.  https://t.co/oohkj6t9cl
0.44 @collinsdw lol. sorry. you being from texas and all... i just assumed the worst. ;-)
0.42 la zoo today 🙈🐨🐭🦇🐼🦊 . . . . #beverlyhills90210 #losangeles #lifestyle #lazoo #picoftheday… https://t.co/d7kfwdd8ai
0.52 remember to get up and move today! move free live free!
0.56 i was supposed to go to the grove su soundtrack launch party but work meetings keepin me tied up 😢 https://t.co/dafbshqime
0.45 come get the 🍌 crazy collabo with @amarim2 🔥 thanks everyone for coming out and rocking wit is… https://t.co/a7bdn1wcsg
0.43 this is fucking disgusting. #blacklivesmatter https://t.co/1tjnmjrtxc
0.31 @eliekorio okay that was good bitch



  0%|          | 15/65363 [00:00<24:00, 45.35it/s]

0.48 lisbeth bit me on saturday &amp; this shit still burns smh 🤦🏽‍♂️
0.45 this is another test!   i love my job #makeittalks
0.33 @cerineity but.. but higher education!! contribution to society!! it's a privilege not a debt!!?!! kappa
0.36 those who fight monsters in the dark ... https://t.co/nxzy9dncr2
0.38 flashsale #contour body wraps w/ a #detox cleanse💚@ https://t.co/uvofrsexu1 ends #tonight @10pm #pst - dont miss out. #summer #beverlyhills
0.42 stand by myanmar #makeamovieforeign
0.46 don't let #hardball fool you. william f buckley jr was a racist chauvinist asshole. he was outspoken but rude and narrow-minded.
0.41 i wonder what his reaction to chivas winning 😂 https://t.co/whlvguum0a
0.3 @kidp1casso that would be awesome! i'll have to see if i can do an all over print on sweatshirts 🤔



  0%|          | 25/65363 [00:00<24:39, 44.15it/s]

...

 35%|███▍      | 22849/65363 [08:55<16:29, 42.98it/s]

0.0 damn i wish i was in florida to see the launch today. all good though, will see the next closest thing today!
0.47 .@araksya hey, stand next to @laurensivan on camera so we can see who is actually a more-miniature giraffe.  srsly.    please?
0.46 thank goodness for @lucidchart. ability to work through a software concept without wanting to strangle e/other is p… https://t.co/4p9oxhjmty
0.17 define "art" , @kathygriffin &amp; @cnnopinion . this isn't a new type of image. why the hype? https://t.co/m5lbs0ht9m
0.47 who the fuck wakes up someone at 7 in the fucking morning to bitch at me ?
0.18 she gorgeous https://t.co/4dxsuhe9y9
0.45 @sheymariee dude shut up
0.57 @leonardcohen2 @thetylerwest @sharonmccutchan @balind8 it's utter bullshit. if al gore is so worried about it why d… https://t.co/5pxbzs93nf
0.34 this was me 🙈 https://t.co/huwq0o3usa


 35%|███▍      | 22854/65363 [08:55<16:16, 43.53it/s]

0.37 good morning penny have a blessed day https://t.co/lzqbnhpgno
0.37 last day of high school 😱😭
0.49 is @martinkeown5 on wenger's payroll @piersmorgan?! saying some deluded shit. @alexis_sanchez is 1000x more important than wenger #arsenal
0.45 kd out here making me cry on my plane before it even takes off  #doitforthebay https://t.co/vwpjrs9v6t
0.43 @actalley thanks so much, act!


KeyboardInterrupt: 

In [240]:
df[['screen_name','tweet_content','cleaned_tweet','music','location','trump','immigrant','climate']].\
    sort(columns='music', axis=0,ascending=False)

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,screen_name,tweet_content,cleaned_tweet,music,location,trump,immigrant,climate
44776,brandon_beatnet,bbq music. https://t.co/ldv6ptl2dd,bbq music.,0.869,"chino, ca",False,False,False
27826,Movie_Fiend_69,@flamingods @flying_vinyl incredible music ht...,incredible music,0.867,"norwalk, ca",False,False,False
43535,pressroomsix,4 the best @all4themusiccom in social media f...,4 the best in social media for musicians music...,0.812,"los angeles, ca",False,False,False
57687,JamesPaulRowlan,https://t.co/xv9dr6mi4r james paul rowland ceo...,james paul rowland ceo dope gurl music just…,0.796,"los angeles, ca",False,False,False
5142,ImGus95,listening to this guy and his jazz music,listening to this guy and his jazz music,0.776,santa monica pier,False,False,False
55124,garnett_elvis,love jazz inflected sound vibrations. https://...,love jazz inflected sound vibrations.,0.768,"los angeles, ca",False,False,False
34133,holysoulmakave1,smoking bluntz bumping music,smoking bluntz bumping music,0.768,"lakewood, ca",False,False,False
2172,dances,we only listen to dance music murder rap or t...,listen dance music murder rap good pop music,0.756,"los angeles, ca",False,False,False
23,sonmi,christmas music all day,christmas music day,0.750,"los angeles, ca",False,False,False
54535,SadGirl94z,all i listen to is soulful music https://t.co/...,all i listen to is soulful music,0.748,"inglewood, ca",False,False,False


In [248]:
df[['screen_name','tweet_content','cleaned_tweet','music','location','trump','immigrant','climate']][df['music'] > 0.50].\
sort_values(by='music', axis=0,ascending=False)

,screen_name,tweet_content,cleaned_tweet,music,location,trump,immigrant,climate
44776,brandon_beatnet,bbq music. https://t.co/ldv6ptl2dd,bbq music.,0.869,"chino, ca",False,False,False
27826,Movie_Fiend_69,@flamingods @flying_vinyl incredible music ht...,incredible music,0.867,"norwalk, ca",False,False,False
43535,pressroomsix,4 the best @all4themusiccom in social media f...,4 the best in social media for musicians music...,0.812,"los angeles, ca",False,False,False
57687,JamesPaulRowlan,https://t.co/xv9dr6mi4r james paul rowland ceo...,james paul rowland ceo dope gurl music just…,0.796,"los angeles, ca",False,False,False
5142,ImGus95,listening to this guy and his jazz music,listening to this guy and his jazz music,0.776,santa monica pier,False,False,False
55124,garnett_elvis,love jazz inflected sound vibrations. https://...,love jazz inflected sound vibrations.,0.768,"los angeles, ca",False,False,False
34133,holysoulmakave1,smoking bluntz bumping music,smoking bluntz bumping music,0.768,"lakewood, ca",False,False,False
2172,dances,we only listen to dance music murder rap or t...,listen dance music murder rap good pop music,0.756,"los angeles, ca",False,False,False
23,sonmi,christmas music all day,christmas music day,0.750,"los angeles, ca",False,False,False
54535,SadGirl94z,all i listen to is soulful music https://t.co/...,all i listen to is soulful music,0.748,"inglewood, ca",False,False,False


#### Use LSA to test on similarity on 2 categories

In [ ]:
'business','govt','movies','sports','tech','tv',